# Decision Tree to classify the priority of an incident

The Incident Management dataset has about 141712 records of 24918 incidents. Each state of the incident is being captured as an individual record with few exceptions where the closed state of an incident is recorded more than once. With the help of the below segment of the code, we load and clean the Incident Management data so that only one record representing the truly closed state per incident is obtained.

------------------------------------------------------------------------------------------------------------------------------

##### Create a spark session and load the Incident Management Data set

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('IMMLDDT').getOrCreate()

In [0]:
df = spark.read.csv('dbfs:/FileStore/tables/incident_event_log_reduced.csv',inferSchema=True,header=True)

In [0]:
print((df.count(), len(df.columns)))

(24918, 36)

##### Data pre-processing

In [0]:
# The data set has multiple states(New, Active, Awaiting user info, Resolved, Closed etc. ) of an incident. With the help 
# of the below command, we are just filtering one record per incident, that has the truly closed state of the incident. 

df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

In [0]:
df_unique_incidents.count()

Out[6]: 24918

In [0]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to make predictions
data=df_unique_incidents.select(['caller_id','opened_by','location','category','subcategory',
                                 'u_symptom','assignment_group','priority'])

In [0]:
data=data.dropna()

In [0]:
# Create a 70-30 train test split
train_data,test_data=data.randomSplit([0.7,0.3])

------------------------------------------------------------------------------------------------------------------------------

### Building the Decision Tree Classifier

In [0]:
# Import the required libraries

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [0]:
# Use StringIndexer to convert the categorical columns to hold numerical data

caller_id_indexer = StringIndexer(inputCol='caller_id',outputCol='caller_id_index',handleInvalid='keep')
opened_by_indexer = StringIndexer(inputCol='opened_by',outputCol='opened_by_index',handleInvalid='keep')
location_indexer = StringIndexer(inputCol='location',outputCol='location_index',handleInvalid='keep')
category_indexer = StringIndexer(inputCol='category',outputCol='category_index',handleInvalid='keep')
subcategory_indexer = StringIndexer(inputCol='subcategory',outputCol='subcategory_index',handleInvalid='keep')
u_symptom_indexer = StringIndexer(inputCol='u_symptom',outputCol='u_symptom_index',handleInvalid='keep')
assignment_group_indexer = StringIndexer(inputCol='assignment_group',outputCol='assignment_group_index',handleInvalid='keep')
priority_indexer = StringIndexer(inputCol='priority',outputCol='priority_index',handleInvalid='keep')

In [0]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['caller_id_index','opened_by_index','location_index','category_index',
                                       'subcategory_index','u_symptom_index','assignment_group_index'],
                            outputCol="features")

In [0]:
# Create an object for the Logistic Regression model
# Use the parameter maxBins and assign a value that is equal to or more than the number of categories in any sigle feature

dt_model = DecisionTreeClassifier(labelCol='priority_index',maxBins=5000)

In [0]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[caller_id_indexer,opened_by_indexer,location_indexer,category_indexer,subcategory_indexer,
                        u_symptom_indexer,assignment_group_indexer,priority_indexer,assembler,dt_model])

In [0]:
# It took 8 minutes for this step to execute

fit_model=pipe.fit(train_data)

In [0]:
# Store the results in a dataframe

results = fit_model.transform(test_data)

In [0]:
results.show()

+-----------+--------------+------------+-----------+---------------+-----------+----------------+------------+---------------+---------------+--------------+--------------+-----------------+---------------+----------------------+--------------+--------------------+--------------------+--------------------+----------+
 caller_id| opened_by| location| category| subcategory| u_symptom|assignment_group| priority|caller_id_index|opened_by_index|location_index|category_index|subcategory_index|u_symptom_index|assignment_group_index|priority_index| features| rawPrediction| probability|prediction|
+-----------+--------------+------------+-----------+---------------+-----------+----------------+------------+---------------+---------------+--------------+--------------+-----------------+---------------+----------------------+--------------+--------------------+--------------------+--------------------+----------+
Caller 1045|Opened by 501|Location 204|Category 21|Subcategory 177|Symptom 494| Group 70|3 - Moderate| 497.0| 8.0| 0.0| 36.0| 46.0| 23.0| 0.0| 0.0|[497.0,8.0,0.0,36...|[1079.0,34.0,46.0...|[0.90444258172673...| 0.0|
Caller 1054| Opened by 24|Location 143|Category 32| Subcategory 9|Symptom 105| Group 70|3 - Moderate| 1382.0| 1.0| 2.0| 4.0| 4.0| 11.0| 0.0| 0.0|[1382.0,1.0,2.0,4...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
 Caller 113|Opened by 390|Location 161|Category 23| Subcategory 3| ?| Group 62|3 - Moderate| 4385.0| 17.0| 1.0| 8.0| 14.0| 1.0| 47.0| 0.0|[4385.0,17.0,1.0,...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 1213| Opened by 17|Location 204|Category 37|Subcategory 123|Symptom 491| Group 46|3 - Moderate| 4693.0| 0.0| 0.0| 5.0| 12.0| 0.0| 28.0| 0.0|[4693.0,0.0,0.0,5...|[129.0,11.0,79.0,...|[0.49425287356321...| 0.0|
Caller 1263| Opened by 17|Location 143|Category 57|Subcategory 174|Symptom 491| Group 23|3 - Moderate| 454.0| 0.0| 2.0| 9.0| 0.0| 0.0| 5.0| 0.0|[454.0,0.0,2.0,9....|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 1289|Opened by 122|Location 111|Category 53|Subcategory 174|Symptom 491| Group 56|3 - Moderate| 2044.0| 137.0| 9.0| 2.0| 0.0| 0.0| 27.0| 0.0|[2044.0,137.0,9.0...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 1341|Opened by 501|Location 143|Category 26|Subcategory 175| ?| Group 70|3 - Moderate| 2945.0| 8.0| 2.0| 1.0| 2.0| 1.0| 0.0| 0.0|[2945.0,8.0,2.0,1...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 1979|Opened by 247|Location 229| Category 9|Subcategory 174| ?| Group 24|3 - Moderate| 1120.0| 40.0| 18.0| 6.0| 0.0| 1.0| 4.0| 0.0|[1120.0,40.0,18.0...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 2134| Opened by 94|Location 108|Category 57|Subcategory 174|Symptom 491| Group 70|3 - Moderate| 4693.0| 21.0| 3.0| 9.0| 0.0| 0.0| 0.0| 0.0|[4693.0,21.0,3.0,...|[640.0,50.0,49.0,...|[0.84210526315789...| 0.0|
Caller 2272| Opened by 17|Location 161|Category 61| Subcategory 36|Symptom 491| Group 73|3 - Moderate| 2100.0| 0.0| 1.0| 10.0| 8.0| 0.0| 7.0| 0.0|[2100.0,0.0,1.0,1...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 2457| Opened by 24|Location 242| Category 9|Subcategory 164|Symptom 571| Group 70|3 - Moderate| 400.0| 1.0| 47.0| 6.0| 3.0| 8.0| 0.0| 0.0|[400.0,1.0,47.0,6...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 2549| Opened by 17|Location 161|Category 23| Subcategory 3|Symptom 491| ?|3 - Moderate| 3399.0| 0.0| 1.0| 8.0| 14.0| 0.0| 1.0| 0.0|[3399.0,0.0,1.0,8...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 2785| Opened by 17|Location 204|Category 37|Subcategory 135|Symptom 127| ?|3 - Moderate| 1519.0| 0.0| 0.0| 5.0| 6.0| 17.0| 1.0| 0.0|[1519.0,0.0,0.0,5...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 2875| Opened by 58|Location 204|Category 46|Subcategory 223|Symptom 592| Group 39|3 - Moderate| 4243.0| 16.0| 0.0| 3.0| 1.0| 29.0| 3.0| 0.0|[4243.0,16.0,0.0,...|[13684.0,14.0,26....|[0.99606929684087...| 0.0|
Caller 3247| Opened by 8|Location 161|Category 55|Subcategory 185|Symptom 387| Group 24|3 - Moderate| 91.0| 10.0|

In [0]:
results.select(['priority_index','prediction']).show()

+--------------+----------+
priority_index|prediction|
+--------------+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+--------------+----------+
only showing top 20 rows

-------------------------------------------------------------------------------------------------------------------------------

##### Evaluating the model

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="priority_index", predictionCol="prediction", metricName="accuracy")

In [0]:
accuracy = ACC_evaluator.evaluate(results)

In [0]:
print("The accuracy of the decision tree classifier is {}".format(accuracy))

The accuracy of the decision tree classifier is 0.9434959349593496

In [0]:
# from pyspark.mllib.evaluation import BinaryClassificationMetrics

# class CurveMetrics(BinaryClassificationMetrics):
#     def __init__(self, *args):
#         super(CurveMetrics, self).__init__(*args)

#     def _to_list(self, rdd):
#         points = []
#         # Note this collect could be inefficient for large datasets 
#         # considering there may be one probability per datapoint (at most)
#         # The Scala version takes a numBins parameter, 
#         # but it doesn't seem possible to pass this from Python to Java
#         for row in rdd.collect():
#             # Results are returned as type scala.Tuple2, 
#             # which doesn't appear to have a py4j mapping
#             points += [(float(row._1()), float(row._2()))]
#         return points

#     def get_curve(self, method):
#         rdd = getattr(self._java_model, method)().toJavaRDD()
#         return self._to_list(rdd)

In [0]:
# preds = results.select('priority_index','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['label'])))
# points = CurveMetrics(preds).get_curve('roc')

# plt.figure()
# x_val = [x[0] for x in points]
# y_val = [x[1] for x in points]
# plt.title(title)
# plt.xlabel(xlabel)
# plt.ylabel(ylabel)
# plt.plot(x_val, y_val)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-699501117654472> in <module> 
 1 preds = results . select ( 'priority_index' , 'probability' ) . rdd . map ( lambda row : ( float ( row [ 'probability' ] [ 1 ] ) , float ( row [ 'label' ] ) ) ) 
 ----> 2 points = CurveMetrics ( preds ) . get_curve ( 'roc' ) 
 3 
 4 plt . figure ( ) 
 5 x_val = [ x [ 0 ] for x in points ] 

 <command-699501117654714> in __init__ (self, *args) 
 3 class CurveMetrics ( BinaryClassificationMetrics ) : 
 4 def __init__ ( self , * args ) : 
 ----> 5 super ( CurveMetrics , self ) . __init__ ( * args ) 
 6 
 7 def _to_list ( self , rdd ) : 

 /databricks/spark/python/pyspark/mllib/evaluation.py in __init__ (self, scoreAndLabels) 
 56 sc = scoreAndLabels . ctx
 57 sql_ctx = SQLContext . getOrCreate ( sc ) 
 ---> 58 numCol = len ( scoreAndLabels . first ( ) ) 
 59 schema = StructType([
 60 StructField ( "score" , DoubleType ( ) , nullable = False ) , 

 /databricks/spark/python/pyspark/rdd.py in first (self) 
 1491 ValueError : RDD is empty
 1492 """
 -> 1493 rs = self . take ( 1 ) 
 1494 if rs : 
 1495 return rs [ 0 ] 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1473 
 1474 p = range ( partsScanned , min ( partsScanned + numPartsToTry , totalParts ) ) 
 -> 1475 res = self . context . runJob ( self , takeUpToNumLeft , p ) 
 1476 
 1477 items += res

 /databricks/spark/python/pyspark/context.py in runJob (self, rdd, partitionFunc, partitions, allowLocal) 
 1225 finally : 
 1226 os . remove ( filename ) 
 -> 1227 sock_info = self . _jvm . PythonRDD . runJob ( self . _jsc . sc ( ) , mappedRDD . _jrdd , partitions ) 
 1228 return list ( _load_from_socket ( sock_info , mappedRDD . _jrdd_deserializer ) ) 
 1229 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 366.0 failed 1 times, most recent failure: Lost task 0.0 in stage 366.0 (TID 333, ip-10-172-229-132.us-west-2.compute.internal, executor driver): org.apache.spark.api.python.PythonException: 'ValueError: label', from <command-699501117654472>, line 1. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/sql/types.py", line 1582, in __getitem__
 idx = self.__fields__.index(item)
ValueError: 'label' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 654, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 646, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 279, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/rdd.py", line 1469, in takeUpToNumLeft
 yield next(iterator)
 File "/databricks/spark/python/pyspark/util.py", line 109, in wrapper
 return f(*args, **kwargs)
 File "<command-699501117654472>", line 1, in <lambd

In [0]:
# points.show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-699501117654713> in <module> 
 ----> 1 points . show ( ) 

 NameError : name 'points' is not defined

In [0]:
import seaborn as sns